<a href="https://colab.research.google.com/github/zyuzyunda/helper/blob/main/LLM_promting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
pip install yandex-cloud-ml-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 44.0.3 which is incompatible.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.


In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [ ]:
import pandas as pd
import json
import time
import pandas as pd
import time
import json
import requests
import uuid
import base64
from google.colab import userdata

In [ ]:
from google.colab import userdata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.set_option('display.max_colwidth', None)

# Загрузка данных

In [ ]:
df = pd.read_json('/content/drive/My Drive/parser_data/job_annotation_by_role_templateV3.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               5 non-null      int64 
 1   role             5 non-null      object
 2   search_keyword   5 non-null      object
 3   job_description  5 non-null      object
 4   skills           5 non-null      object
 5   experience       5 non-null      object
 6   education        5 non-null      object
dtypes: int64(1), object(6)
memory usage: 412.0+ bytes


In [ ]:
df.head()

,id,role,search_keyword,job_description,skills,experience,education
0,104350075,Product analyst/Продуктовый аналитик (Gamedev),Продуктовый аналитик,<p><strong>FunFlow - аккредитованная IT-компан...,"[sql, python, система визуализации, A/B тестир...","[написание sql-запросов, анализ данных, визуал...","[МГУ, НИУ ВШЭ, СПбГУ]"
1,106622421,Продуктовый аналитик в Кредитные продукты,Продуктовый аналитик,<p><strong>Кредитные продукты</strong> — одна ...,"[Greenplum, Hadoop, Hive, ClickHouse, Tableau,...","[исследование поведения пользователей, поиск т...",[студенты]
2,107321876,Продуктовый аналитик в СБП,Продуктовый аналитик,<strong>Обязанности:</strong> <ul> <li>Создани...,"[Яндекс Метрика, AppMetrica, SQL, Python, NumP...","[создание продуктовой отчетности, внедрение пр...",[курс GoPractice]
3,111522568,Продуктовый аналитик (мобильное приложение),Продуктовый аналитик,<p><strong>БИТРИКС24</strong></p> <ul> <li>20+...,"[SQL, ClickHouse, Athena, Python, Power BI, BI...",[обеспечение измеримости продуктовых изменений...,"[МИФИ, ВШЭ, МГУ, вмк, мех.мат]"
4,112439111,Руководитель группы продуктовых аналитиков (Об...,Продуктовый аналитик,<p>Привет! На связи команда онлайн-школы «Фокс...,"[SQL, ClickHouse, PostgreSQL, Python, Metabase...","[управление командой аналитиков, развитие сотр...",[выпускник университета]


# Промпты

In [ ]:
# Промпты для разных стратегий (zero/few-shot, ru/en)

# version 2.1

prompts = {
    "zero_ru_simple": '''
Извлеките из текста вакансии 3 поля:
- "education" - Образование (вузы, уровни, направления),
- "skills_required" - Обязательные навыки (технологии, методы, soft skills),
- "skills_optional" - необязательные навыки.
- "experience" - (должности, проекты, стаж).

Верните JSON. Каждый пункт — список коротких фраз (1–3 слова).}

''',
################################################
    "few_ru_strict": '''
Вы hr-специалист по извлечению сущностей. Учитывая описание вакансии (может включать HTML-теги), извлеките ровно следующие поля:
- "education": уровень, специальность, вузы.
- "skills_required": только требуемые профессиональные hard/soft skills. Не включайте названия должностей.
- "skills_optional": если явно помечено как "желательно", "будет плюсом".
- "experience": краткие требования к опыту (роли, достижения, стаж).

Игнорируйте HTML-теги. Возвращайте только один JSON-объект с этими тремя ключами.

Формат:
- JSON с ключами в двойных кавычках
- Каждое значение — массив строк по 1–3 слова
- Без пояснений или форматирования

Пример:
{
  "education": ["МГУ", "Прикладная математика", "Магистратура"],
  "skills_required": ["Python", "SQL", "Анализ данных"],
  "skills_optional": ["R", "MySQL", "1C"],
  "experience": ["3 года в аналитике", "Работа с BI", "Разработка дашбордов"]
}
''',
################################################
    "few_ru_skills_split": '''
Извлеките из описания вакансии обязательные поля и верните JSON с ключами:
- "skills_required": hard skills, инструменты, библиотеки,
- "skills_optional": если явно помечено как "желательно", "будет плюсом",
- "experience": упомянутые виды практики, ролей или достижений,
- "education": упомянутые вузы, направления, уровни образования.

Убедитесь, что JSON валиден. Не включайте пояснений. Пример вывода:
{
  "skills_required": ["SQL", "Python", "Tableau"],
  "skills_optional": ["Git", "ClickHouse"],
  "experience": ["3 года в аналитике", "построение дашбордов"],
  "education": ["МГУ", "Экономика"]
}
''',
################################################
    "few_en_hr_style": '''
You are an HR specialist skilled in structured entity extraction. Based on the given job description (may include HTML tags), extract exactly the following fields:

- "education": degrees, universities, programs.
- "skills_required": hard skills, tools, libraries,
- "skills_optional": if it is clearly marked as "desirable", "would be a plus",,
- "experience": roles, durations, and practical expertise expected.

Ignore HTML tags. Return exactly one JSON object with these three keys.

Formatting rules:
- Use double quotes for keys and string values
- Each value must be a JSON array of 1–3 word strings
- Do not include any explanations, comments, or formatting

Example output:
{
  "education": ["Moscow State University", "Applied Mathematics", "Master’s degree"],
  "skills_required": ["Python", "SQL", "Teamwork"],
  "skills_optional": ["Git", "ClickHouse"],
  "experience": ["3 years in analytics", "dashboard development", "working with BI tools"]
  ""
}
'''
}

In [ ]:
# version 2.2
prompts = {
    "zero_ru_simple": f'''
Из описания вакансии сформируйте ровно один JSON с ключами:
"education", "skills_required", "skills_optional", "experience".

Каждое значение — массив строк по 1–3 слова.
Без лишнего текста.

Пример:
{{
  "education": [],
  "skills_required": [],
  "skills_optional": [],
  "experience": []
}}

Описание вакансии:
{{job_description}}
''',

    "few_ru_strict": f'''
Из описания вакансии (возможно с HTML-тегами) извлеките один JSON с ключами:
"education", "skills_required", "skills_optional", "experience".

Формат:
- двойные кавычки для ключей и строк,
- каждое значение — массив из 1–3 слов,
- без комментариев и лишнего текста.

Пример:
{{
  "education": ["МГУ", "Прикладная математика", "Оконченное высшее"],
  "skills_required": ["Python", "SQL", "Анализ данных"],
  "skills_optional": ["R", "MySQL", "1C"],
  "experience": ["3 года в аналитике", "Работа с BI", "Разработка дашбордов"]
}}

Описание вакансии:
{{job_description}}
''',

    "few_ru_skills_split": f'''
Из описания вакансии составьте один JSON с ключами:
"skills_required": обязательные навыки (hard/soft, инструменты, библиотеки),
"skills_optional": помеченные как "желательно", "будет плюсом",
"experience": упомянутые роли, проекты, стаж,
"education": упомянутые вузы, специальности, уровни.

Формат:
- двойные кавычки,
- значения — массивы из 1–3 слов,
- без лишнего текста.

Пример:
{{
  "skills_required": ["SQL", "Python", "Tableau"],
  "skills_optional": ["Git", "ClickHouse"],
  "experience": ["3 года в аналитике", "построение дашбордов"],
  "education": ["МГУ", "Прикладная математика", "Оконченное высшее"]
}}

Описание вакансии:
{{job_description}}
''',

    "few_en_hr_style": f'''
Extract exactly one JSON object with keys:
"education", "skills_required", "skills_optional", "experience".

Rules:
- Use double quotes for keys and string values.
- Each value must be an array of 1–3 word strings in Russian.
- No extra text, comments, or explanations.

Example:
{{
  "education": ["МГУ", "Прикладная математика", "Оконченное высшее"],
  "skills_required": ["Python", "SQL", "Работа в команде"],
  "skills_optional": ["Git", "ClickHouse"],
  "experience": ["3 года в аналитике", "разработка дашбордов", "работа с BI инструментами"]
}}

Job description:
{{job_description}}
'''
}

In [ ]:
# version 3.1

prompts = {

    # --- zero-shot, RU ---
    "zero_ru_simple": f'''
Из описания вакансии верните ровно один JSON-объект с ключами:
"skills", "experience", "education".

• "skills"      – технологии, методы, soft-skills, библиотеки, языки
• "experience"  – роли, стаж, проекты, зоны ответственности
• "education"   – вузы, уровни, программы (может быть пустым массивом)

Правила:
- ключи и строки в двойных кавычках
- каждое значение — массив строк по 1-3 слова
- никакого лишнего текста

Пример:
{{
  "skills": [],
  "experience": [],
  "education": []
}}

Описание вакансии:
{{job_description}}
''',

    # --- few-shot, RU ---
    "few_ru_strict": f'''
Сформируйте JSON с ключами "skills", "experience", "education".

Формат:
- двойные кавычки
- массивы 1-3 слов
- без комментариев

Пример:
{{
  "skills": ["SQL", "математическая статистика", "Power BI"],
  "experience": ["5 лет в BI", "ведение команды"],
  "education": ["МГУ", "Неоконченное высшее"]
}}

Описание:
{{job_description}}
''',

    # --- few-shot, EN-prompt, RU output ---
    "few_en_ru_hr_style": f'''
Extract exactly one JSON object with keys "skills", "experience", "education".

Return **values in Russian**.
Rules: double-quoted keys/strings, arrays of 1-3 word strings, no extra text.

Example:
{{
  "skills": ["ClickHouse", "продуктовые метрики", "Git"],
  "experience": ["3 года аналитики", "создание дашбордов"],
  "education": ["МГУ", "Неоконченное высшее"]
}}

Job description:
{{job_description}}
''',
################################################
    "zero_en_hr_style": '''
You are an HR specialist skilled in structured entity extraction. Based on the given job description (may include HTML tags), extract exactly the following fields:

- "education": degrees, universities, programs.
- "skills": hard skills, tools, libraries,
- "experience": roles, durations, and practical expertise expected.

Ignore HTML tags. Return exactly one JSON object with these three keys.

Formatting rules:
- Use double quotes for keys and string values
- Each value must be a JSON array of 1–3 word strings
- Do not include any explanations, comments, or formatting

Example output:
{
  "education": [],
  "skills": [],
  "experience": []
  ""
}
Job description:
{{job_description}}
''',
    "few_en_hr_style": '''
You are an HR specialist skilled in structured entity extraction. Based on the given job description (may include HTML tags), extract exactly the following fields:

- "Education": degrees, universities, programs.
- "Skills": both hard and soft skills explicitly required.
- "Experience": roles, durations, and practical expertise expected.

Ignore HTML tags. Return exactly one JSON object with these three keys.

Formatting rules:
- Use double quotes for keys and string values
- Each value must be a JSON array of 1–3 word strings
- Do not include any explanations, comments, or formatting

Example output:
{{
  "Education": ["Moscow State University", "Applied Mathematics", "Master’s degree"],
  "Skills": ["Python", "SQL", "Teamwork"],
  "Experience": ["3 years in analytics", "dashboard development", "working with BI tools"]
}}

Job description:
{job_description}
}''',

"few_ru_old": '''
Вы hr-специалист по извлечению сущностей. Учитывая описание вакансии (может включать HTML-теги), извлеките ровно следующие поля:
- "Education": уровень, специальность, вузы.
- "Skills": только требуемые профессиональные hard/soft skills. Не включайте названия должностей.
- "Experience": краткие требования к опыту (роли, достижения, стаж).

Игнорируйте HTML-теги. Возвращайте только один JSON-объект с этими тремя ключами.

Формат:
- JSON с ключами в двойных кавычках
- Каждое значение — массив строк по 1–3 слова
- Без пояснений или форматирования

Пример:
{{
  "Education": ["МГУ", "Прикладная математика", "Магистратура"],
  "Skills": ["Python", "SQL", "Анализ данных"],
  "Experience": ["3 года в аналитике", "Работа с BI", "Разработка дашбордов"]
}}

Описание:
{job_description}
'''
}

In [ ]:
# version 3.2

prompts = {

    # --- zero-shot, RU ---
    "zero_ru": f'''
Из описания вакансии верните ровно один JSON-объект с ключами:
"skills", "experience", "education".

• "skills"      – технологии, методы, soft-skills, библиотеки, языки
• "experience"  – роли, стаж, проекты, зоны ответственности
• "education"   – вузы, уровни, программы (может быть пустым массивом)

Правила:
- ключи и строки в двойных кавычках
- каждое значение — массив строк по 1-3 слова
- никакого лишнего текста

Описание вакансии:
{{job_description}}
''',

    # --- few-shot, RU ---
    "few_ru": f'''
Сформируйте JSON с ключами "skills", "experience", "education".

Формат:
- двойные кавычки
- массивы 1-3 слов
- без комментариев

Пример:
{{
  "skills": ["SQL", "математическая статистика", "Power BI"],
  "experience": ["5 лет в BI", "ведение команды"],
  "education": ["МГУ", "Неоконченное высшее"]
}}

Описание:
{{job_description}}
''',

    "few_en": '''
You are an HR specialist skilled in structured entity extraction. Based on the given job description (may include HTML tags), extract exactly the following fields:

- "Education": degrees, universities, programs.
- "Skills": both hard and soft skills explicitly required.
- "Experience": roles, durations, and practical expertise expected.

Ignore HTML tags. Return exactly one JSON object with these three keys.

Formatting rules:
- Use double quotes for keys and string values
- Each value must be a JSON array of 1–3 word strings
- Do not include any explanations, comments, or formatting

Example output:
{{
  "Education": ["Moscow State University", "Applied Mathematics", "Master’s degree"],
  "Skills": ["Python", "SQL", "Teamwork"],
  "Experience": ["3 years in analytics", "dashboard development", "working with BI tools"]
}}

Job description:
{job_description}
}''',

"few_ru_2": '''
Вы hr-специалист по извлечению сущностей. Учитывая описание вакансии (может включать HTML-теги), извлеките ровно следующие поля:
- "Education": уровень, специальность, вузы.
- "Skills": только требуемые профессиональные hard/soft skills. Не включайте названия должностей.
- "Experience": краткие требования к опыту (роли, достижения, стаж).

Игнорируйте HTML-теги. Возвращайте только один JSON-объект с этими тремя ключами.

Формат:
- JSON с ключами в двойных кавычках
- Каждое значение — массив строк по 1–3 слова
- Без пояснений или форматирования

Пример:
{{
  "Education": ["МГУ", "Прикладная математика", "Магистратура"],
  "Skills": ["Python", "SQL", "Анализ данных"],
  "Experience": ["3 года в аналитике", "Работа с BI", "Разработка дашбордов"]
}}

Описание:
{job_description}
'''
}

In [ ]:
# zero-shot
query1 = f'''
Извлеките из текста вакансии 3 поля:
- Образование (вузы, уровни, направления),
- Навыки (технологии, методы, soft skills),
- Опыт (должности, проекты, стаж).

Верните JSON. Каждый пункт — список коротких фраз (1–3 слова).
Описание:
{job_description}
'''

In [ ]:
# few-shot
query2 = f'''
Вы hr-специалист по извлечению сущностей. Учитывая описание вакансии (может включать HTML-теги), извлеките ровно следующие поля:
- "Образование": уровень, специальность, вузы.
- "Навыки": только требуемые профессиональные hard/soft skills. Не включайте названия должностей.
- "Опыт": краткие требования к опыту (роли, достижения, стаж).

Игнорируйте HTML-теги. Возвращайте только один JSON-объект с этими тремя ключами.

Формат:
- JSON с ключами в двойных кавычках
- Каждое значение — массив строк по 1–3 слова
- Без пояснений или форматирования

Пример:
{{
  "Образование": ["МГУ", "Прикладная математика", "Магистратура"],
  "Навыки": ["Python", "SQL", "Анализ данных"],
  "Опыт": ["3 года в аналитике", "Работа с BI", "Разработка дашбордов"]
}}

Описание:
{job_description}
'''

In [ ]:
# промпт на основе уже извлеченных (few-shot)
query3 = f'''
Извлеките из описания вакансии обязательные поля и верните JSON с ключами:
- "skills_required": hard skills, инструменты, библиотеки,
- "skills_optional": если явно помечено как "желательно", "будет плюсом",
- "experience": упомянутые виды практики, ролей или достижений,
- "education": упомянутые вузы, направления, уровни образования.

Убедитесь, что JSON валиден. Не включайте пояснений. Пример вывода:
{{
  "skills_required": ["SQL", "Python", "Tableau"],
  "skills_optional": ["Git", "ClickHouse"],
  "experience": ["3 года в аналитике", "построение дашбордов"],
  "education": ["МГУ", "Экономика"]
}}
Текст:
{job_description}
'''

In [ ]:
# few-shot + english
query4 = '''
You are an HR specialist skilled in structured entity extraction. Based on the given job description (may include HTML tags), extract exactly the following fields:

- "Education": degrees, universities, programs.
- "Skills": both hard and soft skills explicitly required.
- "Experience": roles, durations, and practical expertise expected.

Ignore HTML tags. Return exactly one JSON object with these three keys.

Formatting rules:
- Use double quotes for keys and string values
- Each value must be a JSON array of 1–3 word strings
- Do not include any explanations, comments, or formatting

Example output:
{{
  "Education": ["Moscow State University", "Applied Mathematics", "Master’s degree"],
  "Skills": ["Python", "SQL", "Teamwork"],
  "Experience": ["3 years in analytics", "dashboard development", "working with BI tools"]
}}

Job description:
{job_description}
'''

# Подключение GigaChat

In [ ]:
# Используем вкладку "Секреты" левой панели Colab для хранения ключей
from google.colab import userdata
client_id = userdata.get('SBER_ID')
secret = userdata.get('SBER_SECRET')
auth = userdata.get('SBER_AUTH')

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [ ]:
credentials = f"{client_id}:{secret}"
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
assert encoded_credentials == auth, "Проблема с авторизацией!"


In [ ]:
# Получение токена
def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    rq_uid = str(uuid.uuid4())
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }
    payload = {'scope': scope}
    response = requests.post(url, headers=headers, data=payload, verify=False)
    return response

response = get_token(auth)
if response.status_code != 200:
    raise Exception("Не удалось получить токен")
giga_token = response.json()['access_token']

In [ ]:
# 2. Функция для отправки запроса в GigaChat

def get_chat_completion(auth_token, user_message):
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
    payload = json.dumps({
        "model": "GigaChat",
        "messages": [{"role": "user", "content": user_message}],
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 512,
        "repetition_penalty": 1,
        "update_interval": 0
    })
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }
    response = requests.post(url, headers=headers, data=payload, verify=False)
    return response


# Подключение YandexGPT

https://habr.com/ru/articles/780008/

In [ ]:
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(
    folder_id="b1goh6tkvs7g0k8ifumb", auth="y0__xC7j9uEBhjB3RMgn4P6_xLSa8lwmfOiLze9WMnDapNPdGBoZQ"
)

model = sdk.models.completions("yandexgpt-lite", model_version="rc")
model = model.configure(temperature=0.56)
result = model.run(
    [
        {"role": "system", "text": "я пошла домой и купила самовар а маша ушла в магазин"},
        {
            "role": "user",
            "text": "Суммаризируй текст, который я напишу в следующем сообщении. Задача понятна?",
        },
    ]
)

for alternative in result:
    print(alternative)

AioRpcError: <AioRpcError of RPC that terminated with:
	code = StatusCode.PERMISSION_DENIED
	details = "Permission denied"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {grpc_message:"Permission denied", grpc_status:7, created_time:"2025-05-02T22:18:29.324344385+00:00"}"
	endpoint = "llm.api.cloud.yandex.net:443"
	stub_class = TextGenerationServiceStub
	x-client-request-id = "5d3ac4cd-3cd4-4c3d-8d6c-91baac8067aa"
>

# Подключение Llama

"mistralai/mixtral-8x7b-instruct" — более стабильно на многоязычном вводе

"anthropic/claude-3-haiku" — хорошо структурирует

"gpt-3.5-turbo" — надёжно справляется с задачей даже в zero-shot

In [ ]:
api_key = userdata.get('LLAMA_key')

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

data = {
    "model": "mistralai/mixtral-8x7b-instruct",
    "messages": [
        {"role": "user", "content": "Какие ключевые навыки требуются продуктовым аналитикам?"}
    ]
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)

print(response.json()['choices'][0]['message']['content'])


 Продуктовым аналитикам необходимы следующие ключевые навыки:

1. Анализ данных и статистика: способность собирать, анализировать и интерпретировать данные, используя статистические методы и средства.
2. Коммуникативные навыки: способность четко и ясно общаться своими находками и рекомендациями, адресованными как технической, так и нетехнической аудитории.
3. Понимание продукта: глубокое понимание целевой аудитории, конкурентов, рынка и бизнес-MODELI продукта.
4. Проектирование экспериментов: способность спроектировать и провести эксперименты для проверки гипотез и изучения пользовательского поведения.
5. Понимание пользовательского опыта (UX): способность определять проблемы пользователей и предлагать решения, улучшающие их опыт использования продукта.
6. Технические навыки: знание технологий, используемых в разработке продукта, таких как языки программирования, системы отслеживания ошибок и средства контроля версий.
7. Требование к качеству: способность устанавливать и соблюдать стан

# Загрузка датасета

In [ ]:
# 3. Загрузка датасета
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/parser_data/HHRU_Объединённый_датасет.csv')

Mounted at /content/drive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2096 entries, 0 to 2095
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2096 non-null   int64  
 1   name            2096 non-null   object 
 2   salary_from     542 non-null    float64
 3   salary_to       373 non-null    float64
 4   currency        644 non-null    object 
 5   company         2096 non-null   object 
 6   city            2096 non-null   object 
 7   experience      2096 non-null   object 
 8   employment      2096 non-null   object 
 9   description     2096 non-null   object 
 10  url             2096 non-null   object 
 11  published_at    2096 non-null   object 
 12  key_skills      2096 non-null   object 
 13  search_keyword  2096 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 229.4+ KB


# Тест GigaChat

In [ ]:
# Выбираем описание вакансии для запроса (например, возьмем 10-ю строку)
job_description = df['description'].iloc[3]

In [ ]:
# Извлекаем навыки для этой вакансии
skills = df.loc[df['description'] == job_description, 'key_skills'].values[0]

# Формируем запрос для GigaChat

# Отправляем запрос в GigaChat
response = get_chat_completion(giga_token, query1)

# Получаем и выводим результат
answer = response.json()
print(answer['choices'][0]['message']['content'])

```json
[
  {
    "Образование": [
      "Вузы",
      "Уровни",
      "Направления"
    ],
    "Навыки": [
      "Гибкие методологии разработки",
      "Современный стек технологий на базе .NET, Docker, Kubernetes",
      "Микросервисная архитектура"
    ],
    "Опыт": [
      "Интервьюирование заказчика",
      "Описание требований в форматах Use Cases и User Stories",
      "Проектирование БД",
      "Понимание принципов микросервисной архитектуры"
    ]
  }
]
```


# Обработка GigaChat полностью

In [ ]:
# 4. Цикл обработки вакансий
outputs = []

for idx, job_description in enumerate(df['description']):
    print(f"\n\u2728 Обработка вакансии {idx+1} из {len(df)}")

    # Формируем промпт
    query = f'''
Вы hr-специалист по извлечению сущностей. Учитывая описание вакансии (может включать HTML-теги), извлеките ровно следующие поля:
«Образование»: список требований к степени, учреждениям, дипломам.
«Навыки»: список требуемых hard и soft skills.
«Опыт»: список требуемого опыта.

Возвращайте только один объект JSON с этими тремя ключами.
Используйте двойные кавычки для ключей и строк.
Каждое значение должно быть массивом JSON из элементов из 1–3 слов.
Не выводите дополнительный текст, пояснения или форматирование.

Пример вывода:
{{
"Образование": ["МГУ", "Компьютерные науки и инженерия", "бакалавриат"],
"Навыки": ["1C", "Python", "Машинное обучение", "Коммуникация", "Работа в команде"],
"Опыт": ["Управление командой", "Помощник менеджера", "Менеджер", "Старший инженер"]
}}
Job description input:
{job_description}
'''

    # Отправка запроса
    try:
        response = get_chat_completion(giga_token, query)
        if response.status_code == 200:
            answer = response.json()
            content = answer['choices'][0]['message']['content']
            outputs.append(content)
        else:
            print(f"Ошибка API: {response.status_code}")
            outputs.append("{}")
    except Exception as e:
        print(f"Ошибка обработки: {str(e)}")
        outputs.append("{}")

    time.sleep(1)  # Пауза между запросами

# 5. Сохраняем результаты

df_output = pd.DataFrame({
    'description': df['description'],
    'extracted_entities': outputs
})

save_path = '/content/drive/My Drive/parser_data/HHRU_extracted_outputs.csv'
df_output.to_csv(save_path, index=False)
print(f"\n\u2705 Сохранено в {save_path}")


✨ Обработка вакансии 1 из 2096
Ошибка API: 401

✨ Обработка вакансии 2 из 2096
Ошибка API: 401

✨ Обработка вакансии 3 из 2096
Ошибка API: 401

✨ Обработка вакансии 4 из 2096
Ошибка API: 401

✨ Обработка вакансии 5 из 2096
Ошибка API: 401

✨ Обработка вакансии 6 из 2096
Ошибка API: 401


KeyboardInterrupt: 

In [ ]:
# После остановки выполнения запускаем ЭТО отдельно:

import pandas as pd

# Сохраняем то, что успело обработаться
df_partial = pd.DataFrame({
    'description': df['description'][:len(outputs)],  # берем столько строк, сколько есть outputs
    'extracted_entities': outputs
})

# Сохраняем в файл
save_path = '/content/drive/My Drive/parser_data/HHRU_extracted_outputs_partial.csv'
df_partial.to_csv(save_path, index=False)

print(f"✅ Сохранили частичные данные: {save_path}")

KeyError: 'company'

In [ ]:
df = pd.read_csv('/content/drive/My Drive/parser_data/HHRU_Объединённый_датасет.csv')

# Загружаем уже обработанные результаты
try:
    df_output = pd.read_csv('/content/drive/My Drive/parser_data/HHRU_extracted_outputs_partial.csv')
    already_done = len(df_output)
    print(f"🔵 Уже обработано {already_done} вакансий.")
    outputs = df_output['extracted_entities'].tolist()
except FileNotFoundError:
    already_done = 0
    outputs = []
    print("🆕 Начинаем с нуля.")

# Обрабатываем оставшиеся
for idx, job_description in enumerate(df['description'][already_done:], start=already_done):
    print(f"\n✨ Обработка вакансии {idx+1} из {len(df)}")

    query = f'''
You are an entity extraction specialist. Given a job description (may include HTML tags), extract exactly the following fields:

  • "Education": list of degree requirements, institutions, diplomas.
  • "Skills": list of required hard and soft skills.
  • "Experience": list of required experiences (action verbs, technologies, years).

Return **only** a single JSON object with those three keys.
– Use double quotes for keys and strings.
– Each value must be a JSON array of 1–3‑word items.
– Do **not** output any extra text, explanation, or formatting.

Example output:
{{
  "Education": ["ABC University", "CGPA 3.00", "Computer Science and Engineering", "BSc"],
  "Skills": ["C", "Python", "R", "Machine Learning", "Communication", "Team Work"],
  "Experience": ["Team Management", "Assistant Manager", "Manager", "Senior Engineer", "AWS"]
}}

Job description input:
{job_description}
'''

    try:
        response = get_chat_completion(giga_token, query)
        if response.status_code == 200:
            answer = response.json()
            content = answer['choices'][0]['message']['content']
            outputs.append(content)
        else:
            print(f"Ошибка API: {response.status_code}")
            outputs.append("{}")
    except Exception as e:
        print(f"Ошибка обработки: {str(e)}")
        outputs.append("{}")

    # Автосохранение после каждой вакансии
    df_temp = pd.DataFrame({
        'description': df['description'][:len(outputs)],
        'extracted_entities': outputs
    })
    df_temp.to_csv('/content/drive/My Drive/parser_data/HHRU_extracted_outputs_partial.csv', index=False)

    time.sleep(10)  # пауза между запросами

# Тестирование итоговой версии

In [ ]:
import pandas as pd
import json
import time
import uuid
import requests
import base64
from google.colab import userdata

In [ ]:
# Получение токена GigaChat
client_id = userdata.get('SBER_ID')
secret = userdata.get('SBER_SECRET')
auth = userdata.get('SBER_AUTH')

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    rq_uid = str(uuid.uuid4())
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }
    payload = {'scope': scope}
    response = requests.post(url, headers=headers, data=payload, verify=False)
    return response.json()['access_token']

giga_token = get_token(auth)

# ----------------  Функция-обёртка GigaChat  ---------------- #
def query_gigachat(prompt: str, job_text: str) -> str:
    """Возвращает content первого choices или пустую строку."""
    msg = f"{prompt}\n\nВот описание вакансии:\n{job_text}"

    payload = {
        "model": "GigaChat",
        "messages": [{"role": "user", "content": msg}],
        "temperature": 0.7,
        "top_p": 0.9,
        "n": 1,
        "stream": False,
        "max_tokens": 600
    }

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {giga_token}"
    }

    r = requests.post(
        "https://gigachat.devices.sberbank.ru/api/v1/chat/completions",
        headers=headers, data=json.dumps(payload), verify=False
    )
    data = r.json()
    return data["choices"][0]["message"]["content"] if "choices" in data else ""


# ----------------  Функция-обёртка LLaMA (OpenRouter)  ---------------- #
llama_key = userdata.get("LLAMA_key")

def query_llama(prompt: str, job_text: str) -> str:
    msg = f"{prompt}\n\nВот описание вакансии:\n{job_text}"

    payload = {
        "model": "mistralai/mixtral-8x7b-instruct",
        "messages": [{"role": "user", "content": msg}]
    }

    r = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {llama_key}",
            "Content-Type": "application/json"
        },
        json=payload
    )
    data = r.json()
    return data["choices"][0]["message"]["content"] if "choices" in data else ""

# Основной цикл: 1 вакансия, несколько моделей и промптов
results = []

for idx in range(len(df)):
    job_text = df.loc[idx, 'job_description']

    for prompt_id, prompt_text in prompts.items():
        # GigaChat
        try:
            g_response = query_gigachat(prompt_text, job_text)
            results.append({
                "id": df.loc[idx, 'id'],
                "model": "GigaChat",
                "prompt": prompt_id,
                "skills_output": g_response
            })
        except Exception as e:
            print(f"GigaChat error on row {idx}, prompt {prompt_id}: {e}")

        # LLaMA
        try:
            l_response = query_llama(prompt_text, job_text)
            results.append({
                "id": df.loc[idx, 'id'],
                "model": "LLaMA",
                "prompt": prompt_id,
                "skills_output": l_response
            })
        except Exception as e:
            print(f"LLaMA error on row {idx}, prompt {prompt_id}: {e}")

# Сохраняем результат
results_df = pd.DataFrame(results)
#results_df.to_csv('/content/drive/My Drive/parser_data/skills_extraction_results.csv', index=False)


In [ ]:
results_df.head(130)

,id,model,prompt,skills_output,parsed
0,104350075,GigaChat,zero_ru,"```json\n{\n ""skills"": [""sql"", ""python"", ""data visualization""],\n ""experience"": [""product analysis"", ""dashboard design"", ""A/B testing""],\n ""education"": [""МГУ"", ""НИУ ВШЭ"", ""СПбГУ""]\n}\n```","{'skills': ['sql', 'python', 'data visualization'], 'experience': ['product analysis', 'dashboard design', 'A/B testing'], 'education': ['МГУ', 'НИУ ВШЭ', 'СПбГУ']}"
1,104350075,LLaMA,zero_ru,"{\n""skills"": [""SQL queries"", ""Python"", ""Data visualization"", ""Mathematical statistics"", ""Teamwork"", ""Project management""],\n""experience"": [""Product analytics"", ""Metrics selection"", ""Experiment design"", ""Performance evaluation""],\n""education"": []\n}","{'skills': ['SQL queries', 'Python', 'Data visualization', 'Mathematical statistics', 'Teamwork', 'Project management'], 'experience': ['Product analytics', 'Metrics selection', 'Experiment design', 'Performance evaluation'], 'education': []}"
2,104350075,GigaChat,few_ru,"{\n ""job_description"": ""<p><strong>FunFlow - аккредитованная IT-компания, разработчик популярных мобильных игр и приложений. Уже более 13 лет мы помогаем миллионам людей скрасить свободное время, создавая топовые развлекательные игры и сервисы по всему миру.</strong></p> <p>Социальный игровой проект, который мы разрабатываем, <strong>растет на +20% каждый год</strong>! Наша аудитория – <strong>более миллиона пользователей</strong> ежедневно, которые совершают десятки тысяч покупок и более 50 000 000 сыгранных игр каждый день.</p> <p><strong>Немного о нашем подходе:</strong></p> <p>Мы <strong>быстро растем и адаптируемся</strong> под меняющийся рынок. <strong>Мыслим целями</strong>, а не задачами, ценим свободу в выражении идей и проактивность. <strong>Не боимся рисковать</strong>, тестируем новые инструменты и гипотезы.</p> <p>Наша команда активно растёт, поэтому мы ищем сотрудников, которые смогут нас усилить и помочь достичь новых высот. Сейчас мы в поиске <strong>компетентного Продуктового аналитика</strong>, который поможет реализовать весь потенциал наших проектов.</p> <p>В компании практикуется бизнес-ориентированная аналитика, направленная на принятие решений. У нас хорошо выстроена структура отдела, аналитическая инфраструктура и процессы внутри команды. Вы сможете напрямую влиять на развитие продуктов компании, рекомендуя решения на этапах от оценки конкретных инициатив до составления глобальных планов развития продукта.</p> <p><strong>Формат работы: </strong><em>full office</em></p> <p><strong>Задачи, которые вас ждут:</strong></p> <ul> <li> <p>Предоставлять аналитические инструменты для команды продукта: подбирать метрики и аудитории для аналитики продукта.</p> </li> <li> <p>Создавать новые внутренние аналитические инструменты и развивать имеющиеся.</p> </li> <li> <p>Сопровождать продуктовые фичи от этапа драфта до запуска в прод:</p> </li> </ul> <ol> <li>Оценивать потенциал фичей.</li> <li>Подготавливать дизайн экспериментов (с А/Б тестами и без них).</li> <li>Проверять результаты экспериментов.</li> <li>Оценивать результаты запуска фичей.</li> </ol> <p><strong>Мы ждем, что у вас есть: </strong></p> <ul",{}
3,104350075,LLaMA,few_ru,"{\n ""skills"": [""SQL"", ""математическая статистика"", ""Python"", ""визуализация данных"", ""теория вероятностей"", ""коммуникация"", ""командная работа""],\n ""experience"": [""3-5 лет опыта в анализе данных"", ""запуск продуктовых фичей"", ""проведение А/B тестов""],\n ""education"": [""ВУЗ высший"", ""математическое или инженерное образование""]\n}","{'skills': ['SQL', 'математическая статистика', 'Python', 'визуализация данных', 'теория вероятностей', 'коммуникация', 'командная работа'], 'experience': ['3-5 лет опыта в анализе данных', 'запуск продуктовых фичей', 'проведение А/B тестов'], 'education': ['ВУЗ высший', 'математическое или инженерное образование']}"
4,104350075,GigaChat,few_en,"{\n ""Education"": [""МГУ"", ""НИУ ВШЭ"", ""СПбГУ""],\n ""Skills"": [""Python"", "

In [ ]:
import json

manual_data = [
    {
        "id": 104350075,
        "model": "ya",
        "prompt": "zero_ru",
        "skills_output": """
        {
            "skills": ["SQL", "Python", "А/Б тесты", "ClickHouse", "PostgreSQL", "управление командой", "продуктовая аналитика", "образовательная аналитика", "агрегирование данных", "бизнес-процессы EdTech"],
            "experience": ["продуктовый аналитик", "управление небольшой командой", "работа с DWH", "разработка метрик", "оценка образовательного продукта", "взаимодействие с бизнесом и разработкой"],
            "education": []
        }
        """
    },
    {
        "id": 106622421,
        "model": "ya",
        "prompt": "zero_ru",
        "skills_output": """
        {
            "skills": ["SQL", "Python", "А/Б тесты", "ClickHouse", "PostgreSQL"],
            "experience": ["Продуктовый аналитик", "Управление небольшой командой", "Работа с DWH", "Разработка метрик", "Оценка образовательного продукта"],
            "education": ["ВУЗы", "Уровни", "Программы"]
        }
        """
    },
    {
        "id": 107321876,
        "model": "ya",
        "prompt": "zero_ru",
        "skills_output": """
        {
            "skills": ["Яндекс Метрика", "AppMetica", "Python", "numpy", "pandas", "matplotlib", "sklearn", "SQL", "BI инструменты", "математическая статистика", "продуктовые метрики", "сквозная аналитика", "системное мышление"],
            "experience": ["аналитика данных", "работа с BI системами", "выстраивание сквозной аналитики", "подготовка отчётности", "автоматизация отчётности", "анализ продуктовых метрик"],
            "education": ["курс GoPractice"]
        }
        """
    },
    {
        "id": 111522568,
        "model": "ya",
        "prompt": "zero_ru",
        "skills_output": """
        {
            "skills": ["Python", "SQL", "ClickHouse", "Athena", "Power BI", "BI системы", "A/B тестирование", "анализ данных", "извлечение данных", "визуализация данных", "Scrum"],
            "experience": ["продуктовый аналитик", "веб-аналитик", "работа с большими данными", "SaaS аналитика", "игровая аналитика", "организация A/B тестов"],
            "education": ["МИФИ", "ВШЭ", "МГУ"]
        }
        """
    },
    {
        "id": 112439111,
        "model": "ya",
        "prompt": "zero_ru",
        "skills_output": """
        {
            "skills": ["SQL", "Python", "А/Б тесты", "ClickHouse", "PostgreSQL", "управление командой", "продуктовая аналитика", "образовательная аналитика", "агрегирование данных", "бизнес-процессы EdTech"],
            "experience": ["продуктовый аналитик", "управление небольшой командой", "работа с DWH", "разработка метрик", "оценка образовательного продукта", "взаимодействие с бизнесом и разработкой"],
            "education": []
        }
        """
    }
]

# Преобразуем и добавим в results_df
manual_df = pd.DataFrame(manual_data)
manual_df["parsed"] = manual_df["skills_output"].apply(lambda x: json.loads(x.strip().strip("`")))

# Объединяем с основным DataFrame
results_df = pd.concat([results_df, manual_df], ignore_index=True)


In [ ]:
df_gold = df.copy()

In [ ]:
import pandas as pd
import json
import re
from difflib import SequenceMatcher

def clean_and_parse_json(text):
    if not isinstance(text, str):
        return {}

    # Находим возможный JSON-блок
    match = re.search(r'{.*}', text, flags=re.DOTALL)
    if not match:
        return {}

    json_candidate = match.group(0)

    # Убираем возможные обёртки с ```json
    json_candidate = re.sub(r'^```json\s*', '', json_candidate.strip(), flags=re.IGNORECASE)
    json_candidate = re.sub(r'\s*```$', '', json_candidate.strip())

    # Пробуем распарсить
    try:
        return json.loads(json_candidate)
    except json.JSONDecodeError:
        return {}


results_df['parsed'] = results_df['skills_output'].apply(clean_and_parse_json)

In [ ]:
manual_fixes = {
    (104350075, 'GigaChat', 'few_ru'): {
        "education": [],
        "skills": ["A/B тестирование", "анализ данных", "SQL", "Python", "продуктовая аналитика"],
        "experience": ["проектирование экспериментов", "оценка фичей", "метрики", "визуализация"]
    },
    (106622421, 'GigaChat', 'few_ru'): {
        "education": [],
        "skills": ["SQL", "Python", "таблицы", "визуализация", "анализ гипотез"],
        "experience": ["1+ год работы аналитиком", "анализ поведения пользователей", "A/B тесты"]
    },
    (107321876, 'GigaChat', 'few_en'): {
        "education": [],
        "skills": ["Яндекс Метрика", "AppMetica", "Python", "SQL", "BI инструменты", "GoPractice"],
        "experience": ["аналитика", "автоматизация отчетности", "продуктовая аналитика", "BI системы", "отчетность для Банка России"]
    },
    (112439111, 'LLaMA', 'few_ru_2'): {
        "education": [],
        "skills": ["А/Б тесты", "SQL", "Python", "библиотеки для big data", "ClickHouse", "PostgreSQL"],
        "experience": ["3+ лет в продуктовой команде", "опыт управления командой", "знание бизнес-процессов EdTech"]
    },
    (106622421, 'LLaMA', 'few_en'): {
        "education": ["Master’s degree"],
        "skills": ["product analysis", "A/B testing", "statistics", "SQL", "Python", "machine learning", "data visualization", "user behavior analysis", "product decision making"],
        "experience": ["1 year in product analysis", "database management", "Jira management", "teamwork", "project management"]
    },
}

# Вставка фиксов вручную
for (jid, model, prompt), fixed in manual_fixes.items():
    mask = (
        (results_df['id'] == jid) &
        (results_df['model'] == model) &
        (results_df['prompt'] == prompt)
    )
    results_df.loc[mask, 'skills_output'] = json.dumps(fixed, ensure_ascii=False)

# После вставки обязательно пересоздать колонку parsed:
results_df['parsed'] = results_df['skills_output'].apply(clean_and_parse_json)


In [ ]:
STOPWORDS = {"и","в","на","с","по","за","для","работа","должность", "опыт"}

def normalize_text(text):
    txt = text.lower()
    txt = re.sub(r"[^\w\s]", "", txt)
    return " ".join(w for w in txt.split() if w not in STOPWORDS)

def extract_content_words(lst):
    return { normalize_text(x) for x in lst if isinstance(x, str) and normalize_text(x) }

# direct-метрики
def compute_direct(true_set, pred_set):
    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)
    prec = tp/(tp+fp) if tp+fp else 0
    rec = tp/(tp+fn) if tp+fn else 0
    f1  = 2*prec*rec/(prec+rec) if prec+rec else 0
    return prec, rec, f1

# fuzzy-метрики
def compute_fuzzy(true_set, pred_set, thresh=0.4):
    tp = 0
    matched = set()
    for p in pred_set:
        best_score, best_t = 0, None
        for t in true_set:
            score = SequenceMatcher(None, p, t).ratio()
            if score > best_score:
                best_score, best_t = score, t
        if best_score >= thresh and best_t:
            tp += 1
            matched.add(best_t)
    fp = len(pred_set) - tp
    fn = len(true_set) - len(matched)
    prec = tp/(tp+fp) if tp+fp else 0
    rec = tp/(tp+fn) if tp+fn else 0
    f1  = 2*prec*rec/(prec+rec) if prec+rec else 0
    return prec, rec, f1

# Собираем оценки
expected_keys = {"skills", "experience", "education"}
records = []

for _, row in results_df.iterrows():
    job_id = row['id']
    parsed = row['parsed']
    struct_ok = set(parsed.keys()) == expected_keys
    gold_row = df_gold[df_gold['id']==job_id].iloc[0]
    for field in expected_keys:
        if struct_ok:
            true_items = extract_content_words(gold_row.get(field, []))
            pred_items = extract_content_words(parsed.get(field, []))
            p_d, r_d, f_d = compute_direct(true_items, pred_items)
            p_f, r_f, f_f = compute_fuzzy(true_items, pred_items)
        else:
            p_d=r_d=f_d=p_f=r_f=f_f=0
        records.append({
            'job_id': job_id,
            'model':   row['model'],
            'prompt':  row['prompt'],
            'field':   field,
            'struct_ok': struct_ok,
            'precision_direct': p_d,
            'recall_direct':    r_d,
            'f1_direct':        f_d,
            'precision_fuzzy':  p_f,
            'recall_fuzzy':     r_f,
            'f1_fuzzy':         f_f
        })

eval_df = pd.DataFrame(records)

# Сводные метрики по модели/промпту/полю
summary = eval_df.groupby(['model','prompt','field'])[
    ['precision_direct','recall_direct','f1_direct',
     'precision_fuzzy','recall_fuzzy','f1_fuzzy','struct_ok']
].mean().reset_index()

print(summary)


       model    prompt       field  precision_direct  recall_direct  \
0   GigaChat    few_en   education          0.000000       0.000000   
1   GigaChat    few_en  experience          0.000000       0.000000   
2   GigaChat    few_en      skills          0.100000       0.042857   
3   GigaChat    few_ru   education          0.200000       0.200000   
4   GigaChat    few_ru  experience          0.020000       0.014286   
5   GigaChat    few_ru      skills          0.392857       0.302474   
6   GigaChat  few_ru_2   education          0.000000       0.000000   
7   GigaChat  few_ru_2  experience          0.000000       0.000000   
8   GigaChat  few_ru_2      skills          0.000000       0.000000   
9   GigaChat   zero_ru   education          0.200000       0.200000   
10  GigaChat   zero_ru  experience          0.033333       0.014286   
11  GigaChat   zero_ru      skills          0.826667       0.253408   
12     LLaMA    few_en   education          0.000000       0.000000   
13    

In [ ]:
results_df

,id,model,prompt,skills_output,parsed
0,104350075,GigaChat,zero_ru,"```json\n{\n ""skills"": [""sql"", ""python"", ""data visualization""],\n ""experience"": [""product analysis"", ""dashboard design"", ""A/B testing""],\n ""education"": [""МГУ"", ""НИУ ВШЭ"", ""СПбГУ""]\n}\n```","{'skills': ['sql', 'python', 'data visualization'], 'experience': ['product analysis', 'dashboard design', 'A/B testing'], 'education': ['МГУ', 'НИУ ВШЭ', 'СПбГУ']}"
1,104350075,LLaMA,zero_ru,"{\n""skills"": [""SQL queries"", ""Python"", ""Data visualization"", ""Mathematical statistics"", ""Teamwork"", ""Project management""],\n""experience"": [""Product analytics"", ""Metrics selection"", ""Experiment design"", ""Performance evaluation""],\n""education"": []\n}","{'skills': ['SQL queries', 'Python', 'Data visualization', 'Mathematical statistics', 'Teamwork', 'Project management'], 'experience': ['Product analytics', 'Metrics selection', 'Experiment design', 'Performance evaluation'], 'education': []}"
2,104350075,GigaChat,few_ru,"{""education"": [], ""skills"": [""A/B тестирование"", ""анализ данных"", ""SQL"", ""Python"", ""продуктовая аналитика""], ""experience"": [""проектирование экспериментов"", ""оценка фичей"", ""метрики"", ""визуализация""]}","{'education': [], 'skills': ['A/B тестирование', 'анализ данных', 'SQL', 'Python', 'продуктовая аналитика'], 'experience': ['проектирование экспериментов', 'оценка фичей', 'метрики', 'визуализация']}"
3,104350075,LLaMA,few_ru,"{\n ""skills"": [""SQL"", ""математическая статистика"", ""Python"", ""визуализация данных"", ""теория вероятностей"", ""коммуникация"", ""командная работа""],\n ""experience"": [""3-5 лет опыта в анализе данных"", ""запуск продуктовых фичей"", ""проведение А/B тестов""],\n ""education"": [""ВУЗ высший"", ""математическое или инженерное образование""]\n}","{'skills': ['SQL', 'математическая статистика', 'Python', 'визуализация данных', 'теория вероятностей', 'коммуникация', 'командная работа'], 'experience': ['3-5 лет опыта в анализе данных', 'запуск продуктовых фичей', 'проведение А/B тестов'], 'education': ['ВУЗ высший', 'математическое или инженерное образование']}"
4,104350075,GigaChat,few_en,"{\n ""Education"": [""МГУ"", ""НИУ ВШЭ"", ""СПбГУ""],\n ""Skills"": [""Python"", ""SQL"", ""Математическая статистика"", ""Визуализация данных"", ""Командная работа""],\n ""Experience"": [""Анализ данных"", ""Разработка внутренних инструментов"", ""Проведение экспериментов"", ""Оценка потенциала фич""]\n}","{'Education': ['МГУ', 'НИУ ВШЭ', 'СПбГУ'], 'Skills': ['Python', 'SQL', 'Математическая статистика', 'Визуализация данных', 'Командная работа'], 'Experience': ['Анализ данных', 'Разработка внутренних инструментов', 'Проведение экспериментов', 'Оценка потенциала фич']}"
5,104350075,LLaMA,few_en,"{\n""Education"": [""МГУ"", ""НИУ ВШЭ"", ""СПбГУ""],\n""Skills"": [""SQL"", ""Python"", ""data analysis"", ""data visualization"", ""statistics"", ""probability theory""],\n""Experience"": [""writing SQL queries"", ""data analysis in Python"", ""building data visualizations"", ""teamwork"", ""product feature development""]\n}","{'Education': ['МГУ', 'НИУ ВШЭ', 'СПбГУ'], 'Skills': ['SQL', 'Python', 'data analysis', 'data visualization', 'statistics', 'probability theory'], 'Experience': ['writing SQL queries', 'data analysis in Python', 'building data visualizations', 'teamwork', 'product feature development']}"
6,104350075,GigaChat,few_ru_2,"```json\n{\n ""Education"": [""МГУ"", ""НИУ ВШЭ"", ""СПбГУ""],\n ""Skills"": [""python"", ""sql"", ""анализ данных"", ""математическая статистика"", ""теория вероятностей""],\n ""Experience"": [""3+ года опыта в аналитике"", ""Опыт создания внутренних аналитических инструментов""]\n}\n```","{'Education': ['МГУ', 'НИУ ВШЭ', 'СПбГУ'], 'Skills': ['python', 'sql', 'анализ данных', 'математическая статистика', 'теория вероятностей'], 'Experience': ['3+ года опыта в аналитике', 'Опыт создания внутренних аналитических инструментов']}"
7,104350075,LLaMA,few_ru_2,

In [ ]:
prompts

{'zero_ru': '\nИз описания вакансии верните ровно один JSON-объект с ключами:\n"skills", "experience", "education".\n\n• "skills"      – технологии, методы, soft-skills, библиотеки, языки\n• "experience"  – роли, стаж, проекты, зоны ответственности\n• "education"   – вузы, уровни, программы (может быть пустым массивом)\n\nПравила:\n- ключи и строки в двойных кавычках\n- каждое значение — массив строк по 1-3 слова\n- никакого лишнего текста\n\nОписание вакансии:\n{job_description}\n',
 'few_ru': '\nСформируйте JSON с ключами "skills", "experience", "education".\n\nФормат:\n- двойные кавычки\n- массивы 1-3 слов\n- без комментариев\n\nПример:\n{\n  "skills": ["SQL", "математическая статистика", "Power BI"],\n  "experience": ["5 лет в BI", "ведение команды"],\n  "education": ["МГУ", "Неоконченное высшее"]\n}\n\nОписание:\n{job_description}\n',
 'few_en': '\nYou are an HR specialist skilled in structured entity extraction. Based on the given job description (may include HTML tags), extract 